In [25]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign

In [5]:
date0='0709_1157'
date1='0709_1557'
date2='0709_1934'
dates=[date0,date1,date2]
plate = 13


In [6]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(get_path(date,plate,True,extension='_full_labeled.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [7]:
nx_graphs=[]
poss=[]
for tab in tabs_labeled:
    nx_graph,pos=generate_nx_graph(tab,labeled=True)
    nx_graphs.append(nx_graph)
    poss.append(pos)

In [8]:
nx_graph_clean=[]
for graph in nx_graphs:
    S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
    len_connected=[len(nx_graph.nodes) for nx_graph in S]
    nx_graph_clean.append(S[np.argmax(len_connected)])

In [405]:
skeletons=[]
for nx_graph in nx_graph_clean:
    skeletons.append(generate_skeleton(nx_graph,dim=(20687, 45554)))

In [13]:
skeleton_transformed0=realign(skeletons[0],skeletons[1])

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1899988 1898277
5955 18
0 0


In [407]:
skeletons=[skeleton_transformed0,skeletons[1],skeletons[2]]

In [50]:
nx_graph_poss=[generate_nx_graph_from_skeleton(skeleton) for skeleton in skeletons]
nx_graphs_aligned=[nx_graph_pos[0] for nx_graph_pos in nx_graph_poss]
poss_aligned=[nx_graph_pos[1] for nx_graph_pos in nx_graph_poss]

In [159]:
xbegin=17000
ybegin=0
xend=21000
yend=2000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])

In [136]:
node_smalls=[]
for i,nx_graph in enumerate(nx_graphs_aligned):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss_aligned[i][node][0]<xend and ybegin<poss_aligned[i][node][1]<yend and nx_graph.degree(node)>=1)])

In [127]:
plot_t_tp1(node_smalls[0],node_smalls[1],poss_aligned[0],poss_aligned[1],skeletons_small[0].todense(),skeletons_small[1].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
anx_graph_t0m2,anx_graph_t0m1,anx_graph_t0=nx_graphs_aligned[0], nx_graphs_aligned[1], nx_graphs_aligned[2]
apos_t0m2,apos_t0m1,apos_t0 = poss_aligned[0], poss_aligned[1], poss_aligned[2]
skeleton_doc_0_thinned,skeleton_doc_1_thinned, skeleton_doc_2_thinned= skeletons[0], skeletons[1], skeletons[2]

In [52]:
apos_t0m1,anx_graph_t0m1,anx_graph_t0m2,corresp_tips=second_identification(anx_graph_t0m2,anx_graph_t0m1,apos_t0m2,apos_t0m1,50)


{1537, 2563, 5128, 4621, 1549, 4623, 2578, 538, 4123, 1581, 4153, 5184, 1092, 2633, 1626, 4700, 2653, 4191, 3169, 3170, 100, 2148, 3201, 5251, 5281, 5284, 5285, 676, 3757, 3249, 1717, 2236, 2251, 207, 2770, 4818, 221, 4333, 1776, 757, 2298, 2818, 2820, 2309, 3848, 4882, 4900, 1324, 2874, 3905, 4943, 4436, 2389, 1888, 2917, 4458, 4468, 2425, 3453, 3975, 1423, 3987, 5015, 922, 5021, 5022, 4001, 939, 4021, 5565, 2494, 2501, 4549, 4552, 975, 5590, 484, 1517, 497, 1522, 1016}


In [53]:
poss_aligned[1]=apos_t0m1
nx_graphs_aligned[1]=anx_graph_t0m1
nx_graphs_aligned[0]=anx_graph_t0m2

In [171]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from PIL import Image
from scipy import sparse
import cv2
from pymatreader import read_mat
from util import get_path
from plotutil import show_im,overlap, show_im_rgb
from extract_graph import generate_graph_tab_from_skeleton,generate_nx_graph_from_skeleton,generate_skeleton
import networkx as nx
from copy import deepcopy
from sparse_util import dilate
from scipy.optimize import minimize
from node_id import orient
def whole_movement_identification(nx_graph_tm1,nx_graph_t,pos_tm1,pos_t,length_id=50):
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    tip_origin={tip : tip  for tip in tips}
    pixels_from_tip={tip : [] for tip in tips}
    for number,tip in enumerate(tips):
#         print('tip',pos_tm1[tip],tip)
        if number%100==0:
            print(number/len(tips))
        mini=np.inf
        for edge in nx_graph_t.edges:
            pixel_list=nx_graph_t.get_edge_data(*edge)['pixel_list']
            if np.linalg.norm(np.array(pixel_list[0])-np.array(pos_tm1[tip]))<=5000:
                distance=np.min(np.linalg.norm(np.array(pixel_list)-np.array(pos_tm1[tip]),axis=1))
                if distance<mini:
                    mini=distance
                    right_edge = edge
        origin = np.array(orient(nx_graph_tm1.get_edge_data(*list(nx_graph_tm1.edges(tip))[0])['pixel_list'],pos_tm1[tip]))
        origin_vector = origin[0]-origin[-1]
        branch=np.array(orient(nx_graph_t.get_edge_data(*right_edge)['pixel_list'],pos_t[right_edge[0]]))
        index_nearest_pixel=np.argmin(np.linalg.norm(branch-np.array(pos_tm1[tip]),axis=1))
        candidate_vector = branch[-1]-branch[0]
        dot_product = np.dot(origin_vector,candidate_vector)
#         if tip==5260:
#             print(list(branch[index_nearest_pixel:]))
#             print(list(branch[:index_nearest_pixel]))
        if dot_product>=0:
            root=right_edge[0]
            next_node=right_edge[1]
            pixels_from_tip[tip]+=list(branch[index_nearest_pixel:])
        else:
            root=right_edge[1]
            next_node=right_edge[0]
            pixels_from_tip[tip]+=list(reversed(list(branch[:index_nearest_pixel])))
        #Could improve the candidate vector by chosing pixel around the forme tip but this identification should be rather unambiguous
        last_node=root
        current_node=next_node
        last_branch=np.array(orient(nx_graph_t.get_edge_data(root,next_node)['pixel_list'],pos_t[current_node]))
        def label_node_recursive(last_node,current_node,corresp_label):
            if not current_node in corresp_label.keys() and not current_node in nx_graph_tm1.nodes:
                corresp_label[current_node]=tip
                pixel_list=nx_graph_t.get_edge_data(last_node,current_node)['pixel_list']
                pixels_from_tip[tip]+=pixel_list
                if nx_graph_t.degree(current_node)>=3:
                    for neighbour_t in nx_graph_t.neighbors(current_node): 
                        if neighbour_t!=last_node:
                            label_node_recursive(current_node,neighbour_t,corresp_label)
        label_node_recursive(last_node,current_node,tip_origin)
    return(tip_origin,pixels_from_tip)

In [172]:
tip_origin_t0m2_t0m1,pixels_from_tip= whole_movement_identification(nx_graphs_aligned[0],nx_graphs_aligned[1],poss_aligned[0],poss_aligned[1])

0.0
0.0544069640914037
0.1088139281828074
0.1632208922742111
0.2176278563656148
0.2720348204570185
0.3264417845484222
0.3808487486398259
0.4352557127312296
0.4896626768226333
0.544069640914037
0.5984766050054406
0.6528835690968444
0.7072905331882481
0.7616974972796517
0.8161044613710555
0.8705114254624592
0.9249183895538629
0.9793253536452666


In [194]:
np.mean(growths)

6.503542234332425

In [197]:
np.std(growths)

21.4756528638058

In [187]:
max(growths)

2499

In [199]:
growths=[len(pixels_from_tip[tip]) for tip in pixels_from_tip.keys() if len(pixels_from_tip[tip])<100]

In [218]:
len(pixels_from_tip[5608])

132

In [219]:
len(pixels_from_tip[3879])

184

In [209]:
[tip for tip in pixels_from_tip.keys() if 200>len(pixels_from_tip[tip])>=100]

[5608,
 3879,
 2741,
 4607,
 4582,
 2369,
 666,
 27,
 5463,
 5136,
 2505,
 433,
 5652,
 1879,
 3412,
 5071]

In [222]:
poss_aligned[0][4607]

array([18128, 11403])

In [223]:
node_problem=4607
pos_problem=poss_aligned[0][node_problem]
xbegin=pos_problem[0]-1000
ybegin=pos_problem[1]-1000
xend=pos_problem[0]+1000
yend=pos_problem[1]+1000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graphs_aligned):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss_aligned[i][node][0]<xend and ybegin<poss_aligned[i][node][1]<yend and nx_graph.degree(node)>=1)])

In [224]:
plot_t_tp1([node_problem],[node_problem],poss_aligned[0],poss_aligned[1],skeletons_small[0].todense(),skeletons_small[1].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [202]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(growths,30)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([1.213e+03, 2.960e+02, 1.360e+02, 6.400e+01, 3.300e+01, 1.700e+01,
        1.100e+01, 4.000e+00, 7.000e+00, 1.000e+00, 1.000e+00, 2.000e+00,
        1.000e+00, 1.000e+00, 5.000e+00, 1.000e+00, 0.000e+00, 2.000e+00,
        2.000e+00, 2.000e+00, 0.000e+00, 1.000e+00, 2.000e+00, 0.000e+00,
        2.000e+00, 1.000e+00, 2.000e+00, 2.000e+00, 4.000e+00, 1.000e+00]),
 array([ 0. ,  3.2,  6.4,  9.6, 12.8, 16. , 19.2, 22.4, 25.6, 28.8, 32. ,
        35.2, 38.4, 41.6, 44.8, 48. , 51.2, 54.4, 57.6, 60.8, 64. , 67.2,
        70.4, 73.6, 76.8, 80. , 83.2, 86.4, 89.6, 92.8, 96. ]),
 <BarContainer object of 30 artists>)

In [238]:
def make_growth_picture(pixels_from_tip,pos,shape=(20700,45600),factor=10):
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for tip in pixels_from_tip.keys():
        growth=len(pixels_from_tip[tip])
        beginx=round(pos[tip][0]/factor-factor//2)
        endx=round(pos[tip][0]/factor+factor//2)
        beginy=round(pos[tip][1]/factor-factor//2)
        endy=round(pos[tip][1]/factor+factor//2)
        if growth<=200:
            final_picture[beginx:endx,beginy:endy]+=growth
#         print(growth,beginx,endx)    
#     for x in range(shape[0]//factor):
#         if x%1==0:
#             print(x/2070)
#         for y in range(shape[1]//factor):
#             beginx = x*factor
#             endx=(x+1)*factor
#             beginy = y*factor
#             endy=(y+1)*factor
#             tips_in_frame = [tip for tip in pixels_from_tip.keys() if (beginx<pos[tip][0]<endx) and (beginy<pos[tip][1]<endy)]
#             #shouls be improved, len is not a good indicator of actual length...
#             growth_in_frame = [len(pixels_from_tip[tip]) for tip in tips_in_frame]
#             final_picture[x,y]=np.mean(growth_in_frame)
    return(final_picture)

In [361]:
def make_growth_picture_per_tip(pixels_from_tip,pos,shape=(20700,45600),factor=10,max_growth=200,min_growth=10,per_tip=True):
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    number_tips = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for tip in pixels_from_tip.keys():
        growth=len(pixels_from_tip[tip])
        x=min(round(pos[tip][0]/factor),shape[0]//factor-1)
        y=min(round(pos[tip][1]/factor),shape[1]//factor-1)
        if growth<=max_growth:
#             print(number_tips)
            if growth>=min_growth:
                number_tips[x,y]+=1
                final_picture[x,y]+=growth
#         print(growth,beginx,endx)    
#     for x in range(shape[0]//factor):
#         if x%1==0:
#             print(x/2070)
#         for y in range(shape[1]//factor):
#             beginx = x*factor
#             endx=(x+1)*factor
#             beginy = y*factor
#             endy=(y+1)*factor
#             tips_in_frame = [tip for tip in pixels_from_tip.keys() if (beginx<pos[tip][0]<endx) and (beginy<pos[tip][1]<endy)]
#             #shouls be improved, len is not a good indicator of actual length...
#             growth_in_frame = [len(pixels_from_tip[tip]) for tip in tips_in_frame]
#             final_picture[x,y]=np.mean(growth_in_frame)
    if per_tip:
        return(final_picture/(number_tips+(number_tips==0).astype(np.int)),number_tips)
    else:
        return(final_picture,number_tips)

In [391]:
def compress_skeleton(skeleton_doc,factor):
    shape=skeleton_doc.shape
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for pixel in skeleton_doc.keys():
        x=min(round(pixel[0]/factor),shape[0]//factor-1)
        y=min(round(pixel[1]/factor),shape[1]//factor-1)
        final_picture[x,y]+=1
    return(final_picture)

In [408]:
final_picture1 = compress_skeleton(skeletons[0],10)
final_picture2 = compress_skeleton(skeletons[1],10)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(final_picture1>0,cmap='gray',interpolation='none')
ax.imshow(final_picture2>0,cmap='jet',alpha=0.5,interpolation='none')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [398]:
final_picture,number_tips = make_growth_picture_per_tip(pixels_from_tip,poss_aligned[0],factor=300,max_growth=300,per_tip=True,min_growth=0)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(final_picture)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [388]:
26*500,14*500

(13000, 7000)

In [389]:
pos_problem=(13000, 7000)
xbegin=pos_problem[0]-2000
ybegin=pos_problem[1]-2000
xend=pos_problem[0]+2000
yend=pos_problem[1]+2000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graphs_aligned):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss_aligned[i][node][0]<xend and ybegin<poss_aligned[i][node][1]<yend and nx_graph.degree(node)==1)])

In [390]:
plot_t_tp1(node_smalls[0],node_smalls[1],poss_aligned[0],poss_aligned[1],skeletons_small[0].todense(),skeletons_small[1].todense(),shift=(xbegin,ybegin))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [244]:
poss_aligned[1][5136]

array([19343, 33314])

In [315]:
len(pixels_from_tip[27])

197